<a href="https://colab.research.google.com/github/TalissaMoura/sounding_earth_with_vit/blob/main/01_embeddings_soundingearth_with_vit_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1- Mount drive and call condacolab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## Download code
!git clone 

In [ ]:
## Run this cell to load/download the datasets.

# #Deselect if you need to download the dataset
# print("--- Start Downloading sounding earth dataset files ---")
# !wget -P ./sounding_earth_vit/datasets https://zenodo.org/records/5600379/files/SoundingEarth.tar
# print("--- Finish Downloading sounding earth dataset files ---")

## Here I'm using the data that is saved on my Drive. This is recommended for more faster results.

print("--- Descompress the sounding earth files --- ")
!mkdir -p ./sounding_earth_with_vit/datasets
!tar -xvf ./drive/MyDrive/datasets/SoundingEarth.tar -C ./sounding_earth_with_vit/datasets
print("--- Finish descompress Sounding Earth files ---")

Streaming output truncated to the last 5000 lines.
SoundingEarth/data/spectrograms/33753.jpg
SoundingEarth/data/spectrograms/27904.jpg
SoundingEarth/data/spectrograms/14299.jpg
SoundingEarth/data/spectrograms/27098.jpg
SoundingEarth/data/spectrograms/115.jpg
SoundingEarth/data/spectrograms/50941.jpg
SoundingEarth/data/spectrograms/54039.jpg
SoundingEarth/data/spectrograms/37882.jpg
SoundingEarth/data/spectrograms/14466.jpg
SoundingEarth/data/spectrograms/28434.jpg
SoundingEarth/data/spectrograms/34164.jpg
SoundingEarth/data/spectrograms/7274.jpg
SoundingEarth/data/spectrograms/29421.jpg
SoundingEarth/data/spectrograms/7665.jpg
SoundingEarth/data/spectrograms/18654.jpg
SoundingEarth/data/spectrograms/11018.jpg
SoundingEarth/data/spectrograms/27891.jpg
SoundingEarth/data/spectrograms/4884.jpg
SoundingEarth/data/spectrograms/55459.jpg
SoundingEarth/data/spectrograms/31294.jpg
SoundingEarth/data/spectrograms/11741.jpg
SoundingEarth/data/spectrograms/39952.jpg
SoundingEarth/data/spectrogram

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://bucket-metadados-modelos.s3.us-east-1.amazonaws.com/envs/sounding_earth-4-Linux-x86_64.sh?response-content-disposition=inline&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEFwaCXNhLWVhc3QtMSJGMEQCIFOxQgG3ni4%2BC9n8shZBvrnWo4A9Cz121GbtfRielPcYAiBKRNhDSl%2BxVsO58ZXzG8%2BK8roHNwSYw3Mz25nOrM9bZyrkAggVEAAaDDExMjQ5OTI2OTM4OSIMr9WCfsiU1Nb75Zm%2BKsECz9h31qpHPCvhPXjHRwgLv%2F3Ei5Ja4OMvA7JMafB%2FbN2kOSjczq%2B31ArLVGbGcpNZc225z4yFguSD7KEZPSNMVBnR8ps9HIfZc5sPwu74EIVGph4w697hlhk0kqkLH%2FQ4Y6%2F0HNT%2BYKbbvEVAxxOaW9wQKtky6fBSEyEG0YfYwOf3VXGMslT%2BxWJIafRksm4LiiFBVNp0ziLpDyOad9aHPQuj9JW1gMcNi0KTDnQtL%2BBnkYN13gBC3%2BLbxnOE3k%2F0gRWXWTTudH6yoRv208kBt3xCmCHFoMBEJ1Qm5DizfzanNK2dVmVjIBDBZol3GUh8%2FZzkV4KGlAYdUWtoK6ZjicvL%2FjF8Z%2FnlJaki220rsWdpyCsarK4k11rkzUd%2FhWJ8Yy7carAmXpuhySBe%2Fims0txX845MJgDEhT1IUU07%2FhVkMK7crq0GOrQCN2dkT2mKx3JKv%2BAXYzr67Ew4xOEbM92AV5OalQ7ADzGKe7ZpSq%2F%2BwmR0WE1DfrPlzpoIWzwxtkVeHaArpMeIpA%2F8CrUlVp5jiN9O5wLXDd3D9a8U3yqgaCpy%2FVnDTY7H0d7XhPgedjp4jiK8dtfeabXRRuzLvGpxj

In [ ]:
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


## 2- Import Libs

1 - First, change our current dir to execution from `/content` to `/content/sounding_earth_with_vit`

In [ ]:
!pwd

/content


In [ ]:
%cd /content/sounding_earth_with_vit

/content/sounding_earth_vit_v2


In [ ]:
#import libs
!conda env update -n base -f ./sounding_earth_with_vit/environment_v4.yml

In [ ]:
from tqdm import tqdm
import numpy as np
import wandb
from datetime import datetime
from pathlib import Path
import torch
import torch.nn as nn
from torch.nn.functional import normalize
from torch.optim.lr_scheduler import StepLR,ExponentialLR
from dataloaders import get_loader
from lib import get_optimizer, get_model, get_loss_function, Metrics
from lib.models import FullModelWrapper
from einops import rearrange
#from lib.evaluation import calculate_embeddings
from config import cfg,state

In [ ]:
print('--configs --')
print(cfg)
print('-- states -- ')
print(state)

--configs --
AudioAugmentationMode: True
AugmentationMode: image
BatchSize: 32
DataRoot: ./datasets/SoundingEarth/data
DataThreads: 2
EarlyStopping: inf
Epochs: 100
ImageEncoder: VisionTransformersEncoder
ImageReducer: ImageMeanPool
LatentDim: 128
LocalizedSampling: False
LossArg: []
LossFunction: TripletLoss
MarginScaling: False
MaxSamples: 1
ModelName: 
Optimizer:
  LearningRate: 0.001
  Name: Adam
RunId: 
SoundEncoder: VisionTransformersEncoder
SoundReducer: SoundMeanPool
-- states -- 
BestEpoch: -10000
BestLoss: inf
BoardIdx: 0
Epoch: 0


In [ ]:
# Improve reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

# For GPU
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

Login in wandb!

In [ ]:
import os
os.environ['WANDB__EXECUTABLE']='/usr/local/bin/python3.10'

In [ ]:
import wandb
!wandb login

wandb: Currently logged in as: talissamoura. Use `wandb login --relogin` to force relogin


## 3 - Functions

In [ ]:
def full_forward(model, key, img, snd, snd_split, points, metrics):
    img = img.to(dev)
    snd = snd.to(dev)
    points = points.to(dev)


    Z_img = model.img_encoder(img)

    if cfg.SoundEncoder == 'VisionTransformersEncoder':
      Z_snd = model.snd_encoder(snd)
    else:
      Z_snd = model.snd_encoder(snd,snd_split)

    loss = model.loss_function(Z_img, Z_snd, points)

    with torch.no_grad():
      Z_img = model.loss_function.distance_transform(Z_img, dim=1)
      Z_snd = model.loss_function.distance_transform(Z_snd, dim=1)

      if len(Z_img.size()) & len(Z_snd.size()) == 3:
        Z_img = rearrange(Z_img, 'i a d -> i a d')
        Z_snd = rearrange(Z_snd, 'i a d -> a i d')
      else:
        Z_img = rearrange(Z_img, '(i a) d -> i a d',a=1)
        Z_snd = rearrange(Z_snd, '(i a) d -> i a d',i=1)

      d_matrix = torch.linalg.norm(Z_img - Z_snd, ord=2, dim=2)

      N = d_matrix.shape[0]
      d_true = torch.mean(d_matrix.diag())
      d_false = (torch.mean(d_matrix) - d_true / N) * (N / (N-1))

      res = {"Loss":loss,'AvgMargin': d_false - d_true,}

      return res

def train_epoch(model, data_loader,metrics,accumulation_steps = None,num_epochs=10):
    state.Epoch += 1
    model.train(True)
    metrics.reset()
    # torch.autograd.set_detect_anomaly(True)
    for iteration, data in enumerate(tqdm(data_loader)):
      if accumulation_steps == None:
        res = full_forward(model, *data, metrics)
        metrics.step(**res)
        opt.zero_grad()
        res['Loss'].backward()
        opt.step()
      else:
        res["Loss"] /= accumulation_steps
        res["AvgMargin"] /= accumulation_steps
        metrics.step(**res)
        res['Loss'].backward()
        if ((iteration + 1) % accumulation_steps == 0) or (iteration+1==len(data_loader)):
             opt.step()
             opt.zero_grad()

    metrics_vals, _  = metrics.evaluate()
    logstr = ', '.join(f'{k}: {v:2f}' for k, v in metrics_vals.items())
    print(f'Epoch {state.Epoch:03d} Trn: {metrics_vals}')
    m = {f'trn/{met}': val for met, val in metrics_vals.items()}
    m['_epoch'] = state.Epoch
    wandb.log(m)

@torch.no_grad()
def val_epoch(model, data_loader, metrics):
    model.train(False)
    metrics.reset()
    for iteration, data in enumerate(tqdm(data_loader)):
        res = full_forward(model, *data, metrics)
        metrics.step(**res)

    metrics_vals, metrics_hist = metrics.evaluate()
    logstr = ', '.join(f'{k}: {v:2f}' for k, v in metrics_vals.items())
    print(f'Epoch {state.Epoch:03d} Val: {metrics_vals}')
    m = {f'val/{met}': val for met, val in metrics_vals.items()}
    m['_epoch'] = state.Epoch
    wandb.log(m)

    # # Save model Checkpoint
    if state.Epoch % 5 == 0:
        torch.save(model.state_dict(), checkpoints / f'{state.Epoch:02d}.pt')
    torch.save(model.state_dict(), checkpoints / 'latest.pt')

    if metrics_vals['Loss'] < state.BestLoss:
        print(f'Saving Checkpoint at Epoch {state.Epoch} as best one yet!')
        state.BestLoss = metrics_vals['Loss']
        state.BestEpoch = state.Epoch
        torch.save(model.state_dict(), checkpoints / f'best.pt')

    return state.BestEpoch + cfg.EarlyStopping < state.Epoch

# @torch.no_grad()
# def calculate_embeddings(model, loader_type, device):
#     loader = get_loader(mode=loader_type, batch_size=32, num_workers=6, max_samples=50)
#     keys, Z_snd, Z_img = [], [], []
#     tf = model.loss_function.distance_transform
#     for key, img, snd, snd_split, distance in tqdm(loader, 'Embeddings'):
#         snd = snd.to(device)
#         img = img.to(device)

#         Z_snd.append(tf(model.snd_encoder(snd, snd_split)))
#         Z_img.append(tf(model.img_encoder(img)))
#         keys.append(key)

#     keys = np.concatenate(keys)
#     Z_img = torch.cat(Z_img)
#     Z_snd = torch.cat(Z_snd)
#     print('Z_img shape: ',Z_img.shape)
#     print('Z_snd shape:', Z_snd.shape)

#     return keys, Z_img, Z_snd

## 4 - Train

In [ ]:
## using CUDA
dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'training on {dev} device')

## init wandb
cfg.defrost()
run_id = 'run_vit_large_patch32_224_ft_in21k_v3-2'
model_id = 'timm/vit_large_patch32_224.orig_in21k'
#run_id = wandb.run.id
cfg.RunId = run_id
cfg.ModelName = model_id
cfg.Epochs = 10
wandb.init(project='test-audiovisual',id=run_id,reinit=True)
cfg.freeze()
wandb.config.update(cfg)


## encoders and models
if cfg.ImageEncoder == 'VisionTransformersEncoder':
  img_encoder = get_model(cfg.ImageEncoder,
        batch_size=cfg.BatchSize, output_dim=cfg.LatentDim,
        model_id=cfg.ModelName
    )
else:
  img_encoder   = get_model(cfg.ImageEncoder, reducer=cfg.ImageReducer,
        input_dim=3, output_dim=cfg.LatentDim, final_pool=False
    )
if cfg.SoundEncoder == 'VisionTransformersEncoder':
  snd_encoder = get_model(cfg.SoundEncoder,
        batch_size=cfg.BatchSize, output_dim=cfg.LatentDim,
        model_id=cfg.ModelName
    )
else:
  snd_encoder   = get_model(cfg.SoundEncoder, reducer=cfg.SoundReducer,
        input_dim=1, output_dim=cfg.LatentDim, final_pool=True
    )

loss_function = get_loss_function(cfg.LossFunction)(*cfg.LossArg)

model = FullModelWrapper(img_encoder, snd_encoder, loss_function).to(dev)

weight_decay = 1e-6  # Adjust the weight decay value as needed
opt = get_optimizer(cfg.Optimizer.Name)(model.parameters(), lr=cfg.Optimizer.LearningRate,
                                        weight_decay=weight_decay)
# Define a learning rate scheduler
scheduler = ExponentialLR(optimizer=opt, gamma=0.2)

# # logs dir
if wandb.run.name:
        log_dir = Path('logs') / wandb.run.name
else:
        log_dir = Path('logs') / datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
log_dir.mkdir(parents=True, exist_ok=False)
checkpoints = log_dir / 'checkpoints'
checkpoints.mkdir()
with open(log_dir / f'config.yml', 'w') as f:
    print(cfg.dump(), file=f)

train_data = get_loader(32, num_workers=12, mode='train', max_samples=1,is_vit_for_audio=True)
val_data = get_loader(32, num_workers=12, mode='val', max_samples=1,is_vit_for_audio=True)

metrics = Metrics()
num_epochs = cfg.Epochs

for epoch in range(num_epochs):
    print(f'Starting epoch "{epoch+1}"')
    train_epoch(model, train_data, metrics,num_epochs=num_epochs,accumulation_steps = None)
    stop_early = val_epoch(model, val_data, metrics)
    scheduler.step()
    if stop_early:
        print(f'Stopping Early after {cfg.EarlyStopping} epochs without improvement')

print(f"Done! after {epoch+1} epochs!!")
wandb.finish()

training on cuda:0 device


wandb: wandb version 0.16.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.5
wandb: Run data is saved locally in /content/sounding_earth_vit_v2/wandb/run-20240120_183955-run_vit_large_patch32_224_ft_in21k_v3-2
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run run_vit_large_patch32_224_ft_in21k_v3-2
wandb: ⭐️ View project at https://wandb.ai/talissamoura/test-audiovisual
wandb: 🚀 View run at https://wandb.ai/talissamoura/test-audiovisual/runs/run_vit_large_patch32_224_ft_in21k_v3-2


Number of Samples: 35757
Number of Samples: 5097
Starting epoch "1"


100%|██████████| 1117/1117 [07:54<00:00,  2.35it/s]


Epoch 001 Trn: {'Loss': 0.03702555224299431, 'AvgMargin': 11.017668724060059}


100%|██████████| 159/159 [00:24<00:00,  6.58it/s]


Epoch 001 Val: {'Loss': 0.03124999813735485, 'AvgMargin': 13.045589447021484}
Saving Checkpoint at Epoch 1 as best one yet!
Starting epoch "2"


  6%|▋         | 71/1117 [00:33<08:16,  2.11it/s]


KeyboardInterrupt: 

In [ ]:
wandb.finish()

wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb: trn/AvgMargin ▁
wandb:      trn/Loss ▁
wandb: val/AvgMargin ▁
wandb:      val/Loss ▁
wandb: 
wandb: Run summary:
wandb: trn/AvgMargin 11.01767
wandb:      trn/Loss 0.03703
wandb: val/AvgMargin 13.04559
wandb:      val/Loss 0.03125
wandb: 
wandb: 🚀 View run run_vit_large_patch32_224_ft_in21k_v3-2 at: https://wandb.ai/talissamoura/test-audiovisual/runs/run_vit_large_patch32_224_ft_in21k_v3-2
wandb: Synced 6 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240120_183955-run_vit_large_patch32_224_ft_in21k_v3-2/logs


In [ ]:
!cp -r ./logs/run_vit_large_patch32_224_ft_in21k_v3  ../drive/MyDrive/envs/sounding_earth/sounding_earth_vit/logs/